# emoFace

run all cells

In [24]:
import cv2,os,random,numpy as np

In [25]:
#getting emoji name list
def getEmojis():
    emojis_folder = 'emojis/'
    emojis = []
    for emoji in range(len(os.listdir(emojis_folder))):
        emojis.append(cv2.imread(emojis_folder+str(emoji)+'.png', -1))
    return emojis

In [26]:
def blend_transparent(face_img, overlay_t_img):
    # Split out the transparency mask from the colour info
    overlay_img = overlay_t_img[:,:,:3] # Grab the BRG planes
    overlay_mask = overlay_t_img[:,:,3:]  # And the alpha plane

    # Again calculate the inverse mask
    background_mask = 255 - overlay_mask

    # Turn the masks into three channel, so we can use them as weights
    overlay_mask = cv2.cvtColor(overlay_mask, cv2.COLOR_GRAY2BGR)
    background_mask = cv2.cvtColor(background_mask, cv2.COLOR_GRAY2BGR)

    # Create a masked out face image, and masked out overlay
    # We convert the images to floating point in range 0.0 - 1.0
    face_part = (face_img * (1 / 255.0)) * (background_mask * (1 / 255.0))
    overlay_part = (overlay_img * (1 / 255.0)) * (overlay_mask * (1 / 255.0))

    # And finally just add them together, and rescale it back to an 8bit integer image    
    return np.uint8(cv2.addWeighted(face_part, 255.0, overlay_part, 255.0, 0.0))


#resizing the the emoji 
def blend(image, emoji, position):
    
    #getting the coordinates
    x, y, w, h = position
    
    #getting the emoji
    emoji = cv2.resize(emoji, (w, h))
    try:
        #resizing the emoji
        image[y:y+h, x:x+w] = blend_transparent(image[y:y+h, x:x+w], emoji)
    except:
        pass
    
    #returning the resized image
    return image


In [27]:
def emoFace():
    
    #get the emoji name list
    emojis = getEmojis()
    
    #getting the face detect cascade    
    faceDetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    
    #setting the video source to internal camera
    #for external camera change the source value
    cam=cv2.VideoCapture(0)
    
    #setting the default emoji name
    r=6
    
    #starting the face detection process
    while(True):
        
        #reading the image
        rett,img=cam.read()
        
        #transforming the color image to grayscale (black and white)
        gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        
        #assigning the face value
        faces=faceDetect.detectMultiScale(gray,1.3,5)
        
        #changeing the emoji randomly when we get "w "
        if (cv2.waitKey(1)==ord("w")):
            r=random.randint(0,len(emojis)-1)
        
        #deploying the image
        for(x,y,w,h) in faces:

            #getting the resized emoji
            img = blend(img, emojis[r], (x, y, w, h))
        
        #starting the camera window
        cv2.imshow("emoFace",img)
        
        #stopping the process by recieving "q"
        if (cv2.waitKey(1)==ord("q")):
            break
    
    #closing the camera window
    cam.release()
    cv2.destroyWindow("emoFace")

In [28]:
#press "q" repeatedly to quit
#press "w" repeatedly to change emoji

#both are case sensitive, always turn off the capsLock
#

emoFace()
